# SBML to Boltzmann

In [ ]:
import re
class sbml2boltzmann:
    def __init__( self, sbml, deltag ):
        self.sbml = sbml
        self.deltag = deltag
        self.ordRE = re.compile('__([0-9]+)__')
    def repl( self, matchobj ):
        return chr(int(matchobj.group(1)))
   
    def unmunge( self, sbmlid, prefix='M_' ):
        return self.ordRE.sub( self.repl, sbmlid[len(prefix):])
   
    def get_met_comp( self, speciesId ):
        s = self.sbml.getSpecies(speciesId)
        if s.getCompartment() == 'CCO__45__CYTOSOL':
            return self.unmunge( s.getId() + ':' + s.getCompartment() )
        else:
            return self.unmunge( s.getId()).replace('[',':').rstrip(']')
       
    def write_compartment( self, filename ):
        with open(filename, 'w') as out:
            i = 1
            for comp in self.sbml.getListOfCompartments():
                out.write('{}\t{}\tliters\t0\tV\n'.format(self.unmunge(comp.getId()), float(i)))
                i +=1
    def write_concentrations( self, filename, volume=1.0e-15, conc_units=1.0e-9, default_conc=1.1111111e-01):
        with open(filename, 'w') as out:
            out.write('VOLUME\t{}\n'.format(volume))
            out.write('CONC_UNITS\t{}\n'.format(conc_units))
            for species in self.sbml.getListOfSpecies():
                if species.getBoundaryCondition():
                    boundary = 'F'
                else:
                    boundary = 'V'
                out.write('{}\t{}\t{}\n'.format(self.get_met_comp(species.getId()), default_conc, boundary))
   
    def getDeltaG( self, rxnName ):
        if rxnName in self.deltag:
            return self.deltag[rxnName]
        else:
            return 0.0
    def write_reactions( self, filename):
        with open(filename, 'w') as out:
            for rxn in self.sbml.getListOfReactions():
                out.write('REACTION\t{}\n'.format(rxn.getName()))
                out.write('LEFT\t{}\n'.format(' + '.join([self.get_met_comp(sr.getSpecies()) 
                                                         for sr in rxn.getListOfReactants()])))
                out.write('RIGHT\t{}\n'.format(' + '.join([self.get_met_comp(sr.getSpecies()) 
                                                         for sr in rxn.getListOfProducts()])))
                out.write('DGZERO\t{}\n'.format(self.getDeltaG(rxn.getName())))
                out.write('DGZERO_UNITS\t{}\n'.format('KCAL/MOL'))
                out.write('//\n')
                         
s2b = sbml2boltzmann(m, deltag['DeltaG(pH7.3)'].to_dict())
s2b.get_met_comp('M_CPD__45__548')
s2b.write_compartment('nc12.cmpts')
s2b.write_concentrations('nc12.concs.in')
s2b.write_reactions('nc12.rxns.dat')

# Boltzmann to COBRA

In [100]:
import cobra, os,re
from fractions import Fraction
def char2ord( matchobj ):
    return '__{}__'.format( ord(matchobj.group(0)) )
class boltzmann2cobra:
    def __init__( self ):
        self.model = cobra.Model()
        self.metabolites = set()
    
    
    def munge( self, Id, prefix='M_', regexp=r'[^A-Za-z0-9_]' ):
        return prefix + re.sub(regexp, char2ord, Id )
           
    def dat2model( self, datfile ):
        attributes = ['REACTION','LEFT','RIGHT','LEFT_COMPARTMENT','RIGHT_COMPARTMENT','ENZYME_LEVEL','NREGULATION','COMMENT', 'COMMENTS','PATHWAY']
        with open(datfile) as dat:
            modelname = os.path.splitext(os.path.basename(datfile))[0]
            self.model.id = self.munge(modelname)
            self.model.name = modelname
            rxns = []
            rxn = {}
            for line in dat:
                if line.strip() != '//' and line[0] != '#':
                    cols = line.strip().split('\t')
                    if len(cols) >= 2:
                        attribute, value = cols[0].strip(), cols[1].strip()
                        if attribute in attributes:
                            if attribute in rxn:
                                rxn[attribute] += value
                            else:
                                rxn[attribute] = value
                    elif cols[0] == 'COMMENT':
                        pass
                    else:
                        print(line)
                        print(rxn)
                elif line.strip() == '//':
                    if 'REACTION' not in rxn:
                        print(rxn)
                    reaction = self.create_reaction( rxn )
                    rxns.append(reaction)
                    rxn = {}
        return self.model
    def parse_mixture( self, rxn ):
        pattern = re.compile(r'([0-9./]+\s+)?(.*)')
        parts = {}
        for side in ['LEFT','RIGHT']:
            compartment = rxn['{}_COMPARTMENT'.format( side )]
            for part in rxn[side].split(' + '):
                m = pattern.search(part.strip())
                if m:
                    if not m.group(1) or m.group(1) == '':
                        stoichiometry = 1
                    elif '.' in m.group(1):
                        stoichiometry = float(m.group(1).strip())
                    else:
                        stoichiometry = int(m.group(1).strip())
                    if side=='LEFT':
                        stoichiometry = -stoichiometry
                    species =  m.group(2).strip()
                    met = cobra.Metabolite("{}:{}".format(species, compartment),name=species, compartment=compartment)
                    if met not in self.model.metabolites:
                        self.model.add_metabolites( [met] )
                    parts[met] =  stoichiometry
        return parts

    def create_reaction( self, rxn, prefix='', regexp='~'):
        reaction  = cobra.Reaction()
        reaction.id = self.munge(rxn['REACTION'], 'R_')
        reaction.name = rxn['REACTION']    
        self.model.add_reactions([reaction])
        reaction.add_metabolites( self.parse_mixture( rxn ))
        return reaction


In [102]:
b2c = boltzmann2cobra()
model = b2c.dat2model('neurospora_pentose_phos.glycolysis.tca.2.dat')
cobra.io.save_json_model(model, 'neurospora_pentose_phos.glycolysis.tca.2.json',sort=True,pretty=True)

In [9]:
!head neurospora_pentose_phos.glycolysis.tca.2.dat

REACTION ME1m
LEFT	(S)-MALATE + NAD+ 
RIGHT	pyruvate + NADH + CO2
LEFT_COMPARTMENT MITOCHONDRIA
RIGHT_COMPARTMENT MITOCHONDRIA
ENZYME_LEVEL	0.0
//
REACTION ME2m
LEFT    (S)-MALATE + NADP+
RIGHT   PYRUVATE + NADPH + CO2
